# ReAct OpenAPI MS Graph Agent

## Install Required Libraries
Before starting, ensure the required libraries are installed:

In [ ]:
!pip install floki-ai

## Import Environment Variables

In [1]:
from floki import OpenAPIReActAgent
from floki.tool.utils import OpenAPISpecParser
from dotenv import load_dotenv
import logging

## Enable Logging

In [2]:
logging.basicConfig(level=logging.INFO)

## Load Environment Variables

In [3]:
load_dotenv()  # take environment variables from .env.

True

## Define MS Graph HTTP Header with Access Token

In [4]:
import requests
import os

def construct_auth_headers():
    CLIENT_ID = os.getenv("AAD_APP_ID")
    CLIENT_SECRET = os.getenv("AAD_APP_CLIENT_SECRET")
    TENANT_ID = os.getenv("TENANT_ID")

    AUTH_URL = f"https://login.microsoftonline.com/{TENANT_ID}/oauth2/v2.0/token"

    # POST
    auth_response = requests.post(AUTH_URL, {
        'grant_type': 'client_credentials',
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'scope': 'https://graph.microsoft.com/.default'
    })

    # convert the response to JSON
    auth_response_data = auth_response.json()

    # save the access token
    access_token = auth_response_data['access_token']

    return {"Authorization": f"Bearer {access_token}"}


# Get API credentials.
headers = construct_auth_headers()

## Process MS Graph Users OpenAPI Spec Remotely

In [5]:
openapi_spec_url = "https://raw.githubusercontent.com/microsoftgraph/msgraph-sdk-powershell/dev/openApiDocs/v1.0/Users.yml"
spec_parser = OpenAPISpecParser.from_url(openapi_spec_url)

In [6]:
spec_parser.endpoints[0]

('GET /users',
 'List properties and relationships of the user objects.',
 Operation(tags=['users.user'], summary='List users', description='List properties and relationships of the user objects.', externalDocs=ExternalDocumentation(description='Find more info here', url='https://learn.microsoft.com/graph/api/intune-mam-user-list?view=graph-rest-1.0'), operationId='user_ListUser', parameters=[Parameter(description='Indicates the requested consistency level. Documentation URL: https://docs.microsoft.com/graph/aad-advanced-queries', required=False, deprecated=False, style='simple', explode=None, param_schema=Schema(title=None, multipleOf=None, maximum=None, exclusiveMaximum=None, minimum=None, exclusiveMinimum=None, maxLength=None, minLength=None, pattern=None, maxItems=None, minItems=None, uniqueItems=None, maxProperties=None, minProperties=None, required=None, enum=None, type=<DataType.STRING: 'string'>, allOf=None, oneOf=None, anyOf=None, schema_not=None, items=None, properties=None, 

In [7]:
len(spec_parser.endpoints)

236

## (Optional) OpenAPI Specification to OpenAI Function Call

In [8]:
from floki.tool.utils.openapi import openapi_spec_to_openai_fn

functions = openapi_spec_to_openai_fn(spec_parser)

In [9]:
functions[0]

{'definition': {'type': 'function',
  'function': {'name': 'user_ListUser',
   'description': 'List properties and relationships of the user objects.',
   'parameters': {'type': 'object',
    'properties': {'params': {'type': 'object',
      'properties': {'$top': {'minimum': 0.0,
        'type': 'integer',
        'description': 'Show only the first n items'},
       '$search': {'type': 'string',
        'description': 'Search items by search phrases'},
       '$filter': {'type': 'string',
        'description': 'Filter items by property values'},
       '$count': {'type': 'boolean', 'description': 'Include count of items'},
       '$orderby': {'uniqueItems': True,
        'type': 'array',
        'items': {'type': 'string'},
        'description': 'Order items by property values'},
       '$select': {'uniqueItems': True,
        'type': 'array',
        'items': {'type': 'string'},
        'description': 'Select properties to be returned'},
       '$expand': {'uniqueItems': True,
   

## Initialize Chroma Vectorstore

### Define Embedding Function

In [10]:
from floki.document.embedder import SentenceTransformerEmbedder

embedding_function = SentenceTransformerEmbedder(
    model="all-MiniLM-L6-v2"
)

INFO:floki.document.embedder.sentence:Loading SentenceTransformer model: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:floki.document.embedder.sentence:Model loaded successfully.


### Initialize Vectorstore

In [11]:
from floki.storage import ChromaVectorStore

api_vector_store = ChromaVectorStore(
    name="api_toolbox",
    embedding_function=embedding_function,
)

INFO:floki.storage.vectorstores.chroma:ChromaVectorStore initialized with collection: api_toolbox


## Define OpenAPI ReAct Agent

In [12]:
AIAgent = OpenAPIReActAgent(
    role = "MS Graph API Assistant",
    spec_parser=spec_parser,
    api_vector_store=api_vector_store,
    auth_header=headers
)

INFO:floki.llm.openai.client.base:Initializing OpenAI client...
INFO:floki.agent.patterns.openapi.react:Setting up VectorToolStore for OpenAPIReActAgent...
INFO:floki.tool.storage.vectorstore:Adding tools to Vector Tool Store.
INFO:floki.document.embedder.sentence:Generating embeddings for 236 input(s).


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

INFO:floki.tool.executor:Tool registered: GetOpenapiDefinition
INFO:floki.tool.executor:Tool registered: OpenApiCallExecutor
INFO:floki.tool.executor:Tool Executor initialized with 2 registered tools.
INFO:floki.agent.base:Constructing system_prompt from agent attributes.
INFO:floki.agent.base:Using system_prompt to create the prompt template.
INFO:floki.agent.base:Pre-filled prompt template with attributes: ['name', 'role', 'goal', 'instructions']


## Inspect Tools Registered in Agent

In [13]:
AIAgent.tool_executor.help

                                                  Available Tools                                                  
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name                 ┃ Description                                ┃ Signature                                   ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ GetOpenapiDefinition │                                            │ GetOpenapiDefinition(user_input: str)       │
│                      │         Get potential APIs for the user to │                                             │
│                      │ use to accompish task.                     │                                             │
│                      │         You have to choose the right one   │                                             │
│                      │ after getting a response.                  │                                             │
│                      │         This tool MUST be used before      │                                             │
│                      │ calling any APIs.                          │                                             │
│                      │                                            │                                             │
│ OpenApiCallExecutor  │                                            │ OpenApiCallExecutor(path_template: str,     │
│                      │         Execute an API call based on       │ method: str, path_params: Dict, data: Dict, │
│                      │ provided parameters and configuration.     │ headers: Optional = None, params: Optional  │
│                      │         It MUST be used after the          │ = None, kwargs: Any)                        │
│                      │ get_openapi_definition to call APIs.       │                                             │
│                      │         Make sure to include the right     │                                             │
│                      │ header values to authenticate to the API   │                                             │
│                      │ if needed.                                 │                                             │
│                      │                                            │                                             │
└──────────────────────┴────────────────────────────────────────────┴─────────────────────────────────────────────┘

## Search for APIs

In [14]:
prompt = "Get information about a user with ID da48bd32-94bd-4263-b23a-5b9820a67fab"
AIAgent.tool_vector_store.get_similar_tools(query_texts=prompt)

INFO:floki.tool.storage.vectorstore:Searching for tools similar to query: Get information about a user with ID da48bd32-94bd-4263-b23a-5b9820a67fab
INFO:floki.document.embedder.sentence:Generating embeddings for 1 input(s).


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'ids': [['006fe777-65a9-452e-9cda-c9845af3bf51',
   '5987353c-e148-4c6d-91f2-4aa7b9adb34c',
   '717427a9-2b23-43d6-983f-d3f0dc938dcd',
   'a49ffecf-58dd-4be9-8798-793d37bcff36']],
 'embeddings': None,
 'documents': [["user.DirectReport_GetCountAsUser: None. Args schema: {'type': 'object', 'properties': {'params': {'type': 'object', 'properties': {'$search': {'type': 'string', 'description': 'Search items by search phrases'}, '$filter': {'type': 'string', 'description': 'Filter items by property values'}}, 'required': []}, 'headers': {'type': 'object', 'properties': {'ConsistencyLevel': {'type': 'string', 'description': 'Indicates the requested consistency level. Documentation URL: https://docs.microsoft.com/graph/aad-advanced-queries'}}, 'required': []}, 'path_params': {'type': 'object', 'properties': {'user-id': {'type': 'string', 'description': 'The unique identifier of user'}}, 'required': ['user-id']}}}",
   "user.registeredDevice_GetCount: None. Args schema: {'type': 'object', 'p

## Run OpenAPI Agent

In [16]:
AIAgent.run(prompt)

INFO:floki.agent.patterns.react.base:Iteration 1/10 started.
INFO:floki.agent.base:Pre-filled prompt template with variables: dict_keys(['chat_history'])
INFO:floki.llm.openai.chat:Invoking ChatCompletion API.


user:
Get information about a user with ID da48bd32-94bd-4263-b23a-5b9820a67fab

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:floki.llm.openai.chat:Chat completion retrieved successfully.
INFO:floki.agent.patterns.react.base:Executing GetOpenapiDefinition with arguments {'user_input': 'get user information by user ID'}
INFO:floki.tool.executor:Attempting to execute tool: GetOpenapiDefinition
INFO:floki.tool.storage.vectorstore:Searching for tools similar to query: ['get user information by user ID']


Thought: To obtain information about a specific user, I need to identify the correct API that provides user data. I'll start by retrieving potential APIs that can provide user information.
Action: {"name": "GetOpenapiDefinition", "arguments": {"user_input": "get user information by user ID"}}


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

INFO:floki.tool.executor:Tool 'OpenApiCallExecutor' executed successfully.
INFO:floki.agent.patterns.react.base:To obtain information about a specific user by their ID, the "user_GetUser" endpoint seems to be the most appropriate as it allows you to read properties and relationships of the user object.

Let's proceed with calling this API.Observation: {'@odata.context': 'https://graph.microsoft.com/v1.0/$metadata#users/$entity', 'businessPhones': [], 'displayName': 'Support user', 'givenName': None, 'jobTitle': None, 'mail': None, 'mobilePhone': None, 'officeLocation': None, 'preferredLanguage': None, 'surname': None, 'userPrincipalName': 'support@blueteamarsenal.onmicrosoft.com', 'id': 'da48bd32-94bd-4263-b23a-5b9820a67fab'}
INFO:floki.agent.patterns.react.base:Iteration 3/10 started.
INFO:floki.agent.base:Pre-filled prompt template with variables: dict_keys(['chat_history'])
INFO:floki.llm.openai.chat:Invoking ChatCompletion API.


Observation: {'@odata.context': 'https://graph.microsoft.com/v1.0/$metadata#users/$entity', 'businessPhones': [], 'displayName': 'Support user', 'givenName': None, 'jobTitle': None, 'mail': None, 'mobilePhone': None, 'officeLocation': None, 'preferredLanguage': None, 'surname': None, 'userPrincipalName': 'support@blueteamarsenal.onmicrosoft.com', 'id': 'da48bd32-94bd-4263-b23a-5b9820a67fab'}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:floki.llm.openai.chat:Chat completion retrieved successfully.
INFO:floki.agent.patterns.react.base:Agent is responding directly.


Thought: The information for the user with ID `da48bd32-94bd-4263-b23a-5b9820a67fab` is as follows:

- **Display Name**: Support user
- **User Principal Name**: support@blueteamarsenal.onmicrosoft.com
- **Business Phones**: None listed
- **Given Name**: Not available
- **Job Title**: Not available
- **Mail**: Not available
- **Mobile Phone**: Not available
- **Office Location**: Not available
- **Preferred Language**: Not available
- **Surname**: Not available

If you need further details or to perform additional operations, let me know!

--------------------------------------------------------------------------------

assistant:
The information for the user with ID `da48bd32-94bd-4263-b23a-5b9820a67fab` is as follows:

- **Display Name**: Support user
- **User Principal Name**: support@blueteamarsenal.onmicrosoft.com
- **Business Phones**: None listed
- **Given Name**: Not available
- **Job Title**: Not available
- **Mail**: Not available
- **Mobile Phone**: Not available
- **Office L

'The information for the user with ID `da48bd32-94bd-4263-b23a-5b9820a67fab` is as follows:\n\n- **Display Name**: Support user\n- **User Principal Name**: support@blueteamarsenal.onmicrosoft.com\n- **Business Phones**: None listed\n- **Given Name**: Not available\n- **Job Title**: Not available\n- **Mail**: Not available\n- **Mobile Phone**: Not available\n- **Office Location**: Not available\n- **Preferred Language**: Not available\n- **Surname**: Not available\n\nIf you need further details or to perform additional operations, let me know!'